# Swept source control
## Required instruments
1. RedPitaya 
2. DiCon FiberOptic Switch connected by serial cable

## Setting up hardware
### RedPitaya

https://redpitaya.readthedocs.io/en/latest/appsFeatures/remoteControl/remoteControl.html#python



In [2]:
# Blink test
import sys
import time
import redpitaya_scpi as scpi

rp_s = scpi.scpi('10.138.7.215')

led = 0

print ("Blinking LED["+str(led)+"]")

period = 1 # seconds

for i in range(10):
    time.sleep(period/2.0)
    rp_s.tx_txt('DIG:PIN LED' + str(led) + ',' + str(1))
    time.sleep(period/2.0)
    rp_s.tx_txt('DIG:PIN LED' + str(led) + ',' + str(0))

Blinking LED[0]


In [ ]:
# Acquisition of RF signals
import sys
import redpitaya_scpi as scpi
import matplotlib.pyplot as plot

rp_s = scpi.scpi('10.138.7.215')

# there is an option to select coupling when using SIGNALlab 250-12
# rp_s.tx_txt('ACQ:SOUR1:COUP AC') # enables AC coupling on channel 1

# by default LOW level gain is selected
# rp_s.tx_txt('ACQ:SOUR1:GAIN LV') # user can switch gain using this command

rp_s.tx_txt('ACQ:START')
rp_s.tx_txt('ACQ:TRIG NOW')

while 1:
    rp_s.tx_txt('ACQ:TRIG:STAT?')
    if rp_s.rx_txt() == 'TD':
        break

rp_s.tx_txt('ACQ:SOUR1:DATA?')
buff_string = rp_s.rx_txt()
buff_string = buff_string.strip('{}\n\r').replace("  ", "").split(',')
buff = list(map(float, buff_string))

plot.plot(buff)
plot.ylabel('Voltage')
plot.show()
view rawacquire_trigger_posedge.py

In [34]:
import sys
import redpitaya_scpi as scpi
import matplotlib.pyplot as plt
from time import sleep
import socket

fiber_channel = {
    'rp-f08358.local': 0,
}

ip = {}
rp_s = {}

for (hostname,channel) in fiber_channel.items():
    # Get IP and connect to scpi server
    try:
        ip.update({hostname : socket.gethostbyname(hostname)})        
        rp_s.update({hostname : scpi.scpi(ip[hostname])})
    except:
        print('{} is unavailable: {}'.format(hostname, sys.exc_info()))
    else:
        print('{} has IP {} is connected'.format(hostname, ip[hostname]))

# Measurement parameters
integration_time = 1 # sec
# Modulation parameters
wave_form = 'sine'
freq = 1000
ampl = 0.5
offset = 0.5


# Initializing singal generation and acquisition parameters
for (hostname,rp) in rp_s.items():           
    print('Setting signal generation parameters for {}'.format(hostname))
    rp.tx_txt('GEN:RST')
    rp.tx_txt('SOUR1:FUNC' + str(wave_form).upper())
    rp.tx_txt('SOUR1:FREQ:FIX' + str(freq))
    rp.tx_txt('SOUR1:VOLT ' + str(ampl))
    rp.tx_txt('SOUR1:VOLT:OFFS ' + str(offset))
    
    print('Setting signal acquisition parameters for {}'.format(hostname))
    rp.tx_txt('ACQ:RST')
    rp.tx_txt('ACQ:AVG OFF') # turn off averaging    
    rp.tx_txt('ACQ:TRIG NOW') # set to trigger immediately
    rp.tx_txt('ACQ:SOUR1:COUP DC') # enables DC coupling on channel 1
    rp.tx_txt('ACQ:SOUR1:GAIN HV') # set high voltage gain settings
  
   
    # Acquire signal
    print('turn on output and start acquisition')
    rp.tx_txt('OUTPUT1:STATE ON')
    rp.tx_txt('ACQ:START')
    sleep(integration_time)
    
    print('turn off output and finish acquisition')
    rp.tx_txt('ACQ:STOP')
    rp.tx_txt('OUTPUT1:STATE OFF')
    

    rp.tx_txt('ACQ:SOUR1:DATA?')
    buff_string = rp.rx_txt()
    print(buff_string)
    buff_string = buff_string.strip('{}\n\r').replace("  ", "").split(',')
    buff = list(map(float, buff_string))
    

plt.plot(buff)
plt.ylabel('Input [V]')
plt.show()

# Close connections
for (hostname,channel) in rp_s.items():           
    try:        
        rp_s[hostname].close()
    except:
        print('Could not close {}: {}'.format(hostname, sys.exc_info()))
    else:
        print('{} with IP {} successfully disconnected'.format(hostname, ip[hostname]))

rp-f08358.local has IP 10.138.7.215 is connected
Setting signal generation parameters for rp-f08358.local
Setting signal acquisition parameters for rp-f08358.local
turn on output and start acquisition
turn off output and finish acquisition
ERR!ERR!ERR!{0.0227102,0.0340652,0.0227102,0.0141938,0.036904,-0.00283877,0.00567754,0.00567754,0.0198714,0.0141938,0.0170326,0.0198714,0.0255489,0.0170326,0.0227102,-0.0113551,0.00567754,0.0198714,0.0312265,0.036904,0.0170326,0.0283877,0.00567754,0.0113551,0.00851631,0.0141938,0.0141938,0.0141938,0.0227102,0.0198714,-0.00283877,0.0227102,0.00851631,0.00283877,-0.00567754,0.0113551,0.00851631,0.0198714,0.0283877,0.0255489,0.0255489,0.00567754,0.0312265,0.0198714,0.0340652,0.0397428,0.0170326,-0.00851631,0.0113551,0.0283877,0.00567754,0.0198714,0.00567754,0.00567754,0.00567754,0.0170326,0.00567754,0.0283877,-0.00567754,0.0198714,0.00283877,0.00283877,0.0141938,0.0255489,0.0227102,0.0141938,0.0141938,0.0283877,0.00567754,0,0.00567754,0.0198714,0.005677

ValueError: could not convert string to float: 'ERR!ERR!ERR!{0.0227102'

In [28]:
test = {}

key = 'one'
test.update({key : 1})
print(test)
print(test[key])

{'one': 1}
1
